## Homework 3
1. Input BGR images from webcam.
2. Detect your face, mouth, and eyes. 
3. Input BGRA images from files "mustache.png" and "hat.png" (hint: cv2.imread("mustache.png", cv2.IMREAD_UNCHANGED) to read 4 channels)
4. Perform <b> Alpha Blending </b> to add mustache and hat on the right position and orientation of your face.
5. The overlaid mustache and hat should be translated, rotated , and scaled according to the movement of your face. 
6. Show your output images.
7. Any idea on how to detect face features better? Try it and compare the results. (hint: modules of <i>dlib</i> or <i>mediapipe</i>)
8. (5pts bonus) Open/Close your mouth to toggle mustache on/off.
9. (5pts bonus) Blink your eye to toggle hat on/off.
10. Upload your Jupyter code file (*.ipynb)

In [10]:
import cv2
import numpy as np
import mediapipe as mp

In [ ]:
def list_available_cameras(max_index):
    available_cameras = []
    for i in range(max_index):
        cap = cv2.VideoCapture(i)
        
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()               
    return available_cameras

max_camera_index = 10
available_cameras = list_available_cameras(max_camera_index)

In [12]:
print("Available Cameras:", available_cameras)

Available Cameras: [0, 2]


In [2]:
def resize_overlay(overlay_img, target_width):
    h, w = overlay_img.shape[:2]
    scaling_factor = target_width / float(w)
    return cv2.resize(overlay_img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)

In [3]:
def overlay_image_alpha(img, img_overlay, pos, alpha_mask):
    x, y = pos
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])

    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    for c in range(img.shape[2]):
        img[y1:y2, x1:x2, c] = (alpha_mask[y1o:y2o, x1o:x2o] * img_overlay[y1o:y2o, x1o:x2o, c] +
                                (1.0 - alpha_mask[y1o:y2o, x1o:x2o]) * img[y1:y2, x1:x2, c])


In [5]:
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

mustache_img = cv2.imread("mustache.png", cv2.IMREAD_UNCHANGED)
hat_img = cv2.imread("hat.png", cv2.IMREAD_UNCHANGED)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Error: Could not read frame.")
        continue
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(frame_rgb)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            mustache_width = int(w * 0.6)
            resized_mustache_img = resize_overlay(mustache_img, mustache_width)
            resized_mustache_alpha = resized_mustache_img[:, :, 3] / 255.0
            resized_mustache_img = resized_mustache_img[:, :, :3]

            hat_width = int(w * 1.0)
            resized_hat_img = resize_overlay(hat_img, hat_width)
            resized_hat_alpha = resized_hat_img[:, :, 3] / 255.0
            resized_hat_img = resized_hat_img[:, :, :3]

            mustache_pos = (x + int(w * 0.2), y + int(h * 0.5))
            hat_pos = (x - int(w * 0.25), y - int(h * 0.25))

            overlay_image_alpha(frame, resized_mustache_img, mustache_pos, resized_mustache_alpha)
            overlay_image_alpha(frame, resized_hat_img, hat_pos, resized_hat_alpha)

    cv2.imshow('Result', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 